In [6]:
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 1) Carrega os dados

df = pd.read_csv(
    "dados/MQD_1465_final.csv",
    sep=',',              # separador de colunas
    quoting=csv.QUOTE_ALL,# trata todos os campos como strings entre aspas
    quotechar='"',        # caractere de aspas
    skipinitialspace=True, # remove espaços após as vírgulas,
    encoding='utf-8'     # codificação do arquivo
)

df.columns = df.columns.str.strip() # remove espaços em branco dos nomes das colunas
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

print("Colunas disponíveis:")
print(df.columns.tolist())  


# Verificar o tamanho do arquivo e as primeiras linhas
print("Dimensões do DataFrame:", df.shape)
print("\nPrimeiras linhas:")
print(df.head())

# Verificar nomes das colunas
print("\nColunas disponíveis:")
print(df.columns.tolist())


Colunas disponíveis:
['ItemNumber', 'frase', 'cla_maj_femi', 'qtd_maj_femi', 'cla_maj_masc', 'qtd_maj_masc', 'f1_class', 'f2_class', 'f3_class', 'f4_class', 'm1_class', 'm2_class', 'm3_class', 'm4_class', 'f1_tempo', 'f2_tempo', 'f3_tempo', 'f4_tempo', 'm1_tempo', 'm2_tempo', 'm3_tempo', 'm4_tempo']
Dimensões do DataFrame: (1465, 22)

Primeiras linhas:
   ItemNumber                                              frase cla_maj_femi  \
0           1  Você sabia que o menino que mais vai te dar va...     positiva   
1           2  Hoje com 21 anos, como que por auxilio lá do a...     positiva   
2           3  Nesse momento to evitando ela, e ta me dando u...     negativa   
3           4  Meus sentidos de garota diziam que tinha algum...       neutra   
4           5  Aquela promessa que eu te fiz naquele natal, e...     positiva   

   qtd_maj_femi cla_maj_masc  qtd_maj_masc  f1_class  f2_class  f3_class  \
0             4     positiva             4  positiva  positiva  positiva   
1     

In [7]:
# Diagnostic code for DataFrame inspection
print("1. DataFrame Info:")
print(df.info())

print("\n2. Column names with exact representation:")
for col in df.columns:
    print(f"'{col}' - Length: {len(col)}")

print("\n3. Check for hidden characters:")
print([ord(c) for c in df.columns[1]])  # Should show ASCII values for 'frase'

print("\n4. First few rows of specific columns:")
try:
    print("\nFirst row of 'frase':")
    print(df.iloc[0]['frase'])
except KeyError as e:
    print(f"KeyError when accessing 'frase': {e}")

# Try accessing with index instead of name
print("\n5. Second column (should be 'frase'):")
print(df.iloc[0,1])

1. DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ItemNumber    1465 non-null   int64  
 1   frase         1465 non-null   object 
 2   cla_maj_femi  1465 non-null   object 
 3   qtd_maj_femi  1465 non-null   int64  
 4   cla_maj_masc  1465 non-null   object 
 5   qtd_maj_masc  1465 non-null   int64  
 6   f1_class      1465 non-null   object 
 7   f2_class      1465 non-null   object 
 8   f3_class      1465 non-null   object 
 9   f4_class      1465 non-null   object 
 10  m1_class      1465 non-null   object 
 11  m2_class      1465 non-null   object 
 12  m3_class      1465 non-null   object 
 13  m4_class      1465 non-null   object 
 14  f1_tempo      1465 non-null   float64
 15  f2_tempo      1465 non-null   float64
 16  f3_tempo      1465 non-null   float64
 17  f4_tempo      1465 non-null   float64
 18  m1_tempo 

In [9]:
# Check required columns
required_columns = ['frase', 'cla_maj_masc', 'cla_maj_femi']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print("Missing columns:", missing_columns)
    print("Available columns:", df.columns.tolist())
    raise ValueError(f"Missing required columns: {missing_columns}")

# If all columns exist, proceed with the model
X = df['frase'].values
y_masc = df['cla_maj_masc']
y_femi = df['cla_maj_femi']